In [26]:
import numpy as np
import pandas as pd
import pdb

import LammpsPostProcess as lp
import imp
imp.reload(lp)

<module 'LammpsPostProcess' from '/Users/Home/Desktop/Tmp/txt/git/HeaDef/postprocess/LammpsPostProcess.py'>

In [ ]:
#--- comment out if no parameter is passed as an env. variable
env_vars = !cat .env
for var in env_vars:
    key, value = var.split('=')
    os.environ[key] = value

path = os.environ.get('path') #--- don't comment!
fileName = os.environ.get('fileName')
dmean = float(os.environ.get('dmean'))

In [33]:
#fileName = 'data_init.txt'
#dmean = 10.0

#--- read data file
lmpData = lp.ReadDumpFile( '%s/%s'%(path,fileName ) ) 
lmpData.ReadData()
#--- atom obj
atoms = lp.Atoms( **lmpData.coord_atoms_broken[0].to_dict(orient='series') )
#--- box
box = lp.Box( BoxBounds = lmpData.BoxBounds[0],AddMissing = np.array([0.0,0.0,0.0] ) )


In [29]:
pd.DataFrame(atoms.__dict__)

,x,y,z,id,type
0,0.258138,0.242737,0.251894,1,1
1,2.037498,2.023144,0.267614,2,2
2,2.031325,0.253223,2.019011,3,2
3,0.208169,2.010810,2.020765,4,3
4,3.824685,0.228028,0.294745,5,2
...,...,...,...,...,...
859,14.475045,19.792796,19.889660,860,3
860,18.014280,18.000933,18.041932,861,2
861,19.823891,19.773882,18.023069,862,1
862,19.830889,18.031728,19.816855,863,2


In [24]:
box.CellVector

array([[21.33405216,  0.        ,  0.        ],
       [ 0.        , 21.33405216,  0.        ],
       [ 0.        ,  0.        , 21.33405216]])

In [32]:
def Partition( atoms,box, dmean ):
#    pdb.set_trace()
    #--- grid: tiling mapped box with original size
    (xlin, ylin, zlin), (xv, yv, zv) = lp.GetCubicGrid( box.CellOrigin, 
                                                     box.CellVector, 
                                                     dmean,
                                                     margin = 0.0 * dmean, odd = False )
    xi = np.array(list(zip(xv.flatten(), yv.flatten(), zv.flatten())))
    dvol = (xlin[1]-xlin[0])*(ylin[1]-ylin[0])*(zlin[1]-zlin[0])
    (ny,nx,nz) = xv.shape
    nx -= 1
    ny -= 1
    nz -= 1
    assert nx*ny*nz >= 8, 'decrease division length!'
    print(dmean,nx*ny*nz)

    #--- partition box & assign index to each atom
    wrap = lp.Wrap(atoms,box)
    wrap.WrapCoord() #--- wrap inside
    wrap.Set(atoms)
    assert np.sum(wrap.isInside()) == len(atoms.x)
    wrap.GetDimensionlessCords()
    AtomCellId = (wrap.beta * np.array([nx,ny,nz])).astype(int)
    #--- store in a df
    df = pd.DataFrame(np.c_[pd.DataFrame(atoms.__dict__),AtomCellId],
                         columns=list(pd.DataFrame(atoms.__dict__).keys())+['ix','iy','iz'])
    #--- group & compute p and c
    d = df.groupby(by=['ix','iy','iz']).groups #--- key = cell id, value = list of atoms inside
#     print(len(d))
    assert len(d) == nx*ny*nz, 'empty boxes!'
    keys = d.keys()
#     pdb.set_trace()
    
    #--- output as additional lammps script
    count = 0
    for key in keys:
        sfile=open('ScriptGroup.%s.txt'%count,'w')
        sfile.write('group freeGr id\t')
        atomf = df.iloc[d[key]]
        for i in atomf.id:
            sfile.write('%i\t'%i)
        sfile.write('\ngroup frozGr subtract all freeGr')
        sfile.close()
        count += 1
    

Partition( atoms,box, dmean )

10.0 8
